# Energy Envelope Test for Planetary Modulation

Using the "energy envelope model" as a filter, we systematically examine which fundamental frequencies (signals in the residuals) have their energy truly modulated by the 11-year planetary cycle (fitted SSN). This method (energy envelope of high-frequency signals vs. low-frequency driving force) is physically more interpretable than the "full frequency comb" feature engineering of LGBM. We will no longer be limited to 20-35 days, but will traverse all relevant physical frequency bands (rotation, Rieger, annual, QBO), and quantify their respective "modulation relationship" with the 11-year cycle. The complete code for filtering modulated frequencies is provided below. It will: define a series of physical frequency bands of interest (in days). Loop through 4 models (M8+2, M0+3...). For each model, loop through each frequency band. Core step: extract the energy envelope of that frequency band, and (using a simple LGBM) fit $Y_{envelope} = f(X_{planetary\_fit})$, calculating R² on the test set. Finally, a table is summarized showing the average R² for each frequency band across the four models, telling you which frequencies are "worth" being modulated.

In [1]:
import pandas as pd
import numpy as np
import time
import warnings
import os
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from scipy.signal import butter, sosfiltfilt, hilbert
import lightgbm as lgb  # We use a simple LGBM as a verification model

# Ensure astronomy libraries are available (if LGBM code runs here too)
try:
    import lightgbm as lgb
    print("LightGBM imported successfully.")
except ImportError:
    print("Error: lightgbm library not found.")
    exit()

try:
    # Check scipy (signal processing library)
    import scipy
    print(f"Scipy imported successfully (version: {scipy.__version__}).")
except ImportError:
    print("Error: This script requires the scipy library.")
    print("Please install: pip install scipy")
    exit()

warnings.filterwarnings('ignore')

# --- Core Function 1: Bandpass Filter ---
def bandpass_filter(data, lowcut_days, highcut_days, fs=1.0, order=4):
    """
    Apply zero-phase bandpass filter.
    fs=1.0 means the sampling interval is 1 day.
    """
    # Convert period (days) to frequency (1/day)
    low_freq = 1.0 / highcut_days
    high_freq = 1.0 / lowcut_days
    
    # Nyquist frequency
    nyq = 0.5 * fs
    
    # Normalized frequency
    low = low_freq / nyq
    high = high_freq / nyq
    
    # Ensure frequencies are within valid range
    low = max(low, 1e-9) 
    high = min(high, 0.9999)

    if low >= high:
        print(f"Warning: Invalid frequency range [{low}, {high}]. Skipping filter.")
        return data # Return original data

    # Create filter
    sos = butter(order, [low, high], btype='band', output='sos')
    
    # Apply zero-phase filter
    filtered_data = sosfiltfilt(sos, data)
    return filtered_data

# --- Core Function 2: Calculate Energy Envelope ---
def get_energy_envelope(data, smoothing_window_days=365):
    """
    Calculate the energy envelope of the signal (Hilbert transform) and smooth it.
    """
    # 1. Calculate analytic signal
    analytic_signal = hilbert(data)
    
    # 2. Calculate energy envelope (instantaneous amplitude)
    envelope = np.abs(analytic_signal)
    
    # 3. (Critical) Smooth the envelope to observe its long-term trend
    #    We use a centered rolling mean
    envelope_series = pd.Series(envelope)
    smoothed_envelope = envelope_series.rolling(
        window=smoothing_window_days, 
        center=True, 
        min_periods=int(smoothing_window_days * 0.1) # Allow edges
    ).mean()
    
    return smoothed_envelope

# --- Core Function 3: Run Modulation Test ---
def run_modulation_test(
    data_full, model_name, train_end_date, test_end_date, 
    band_name, freq_band_days, envelope_smoothing_days=365
):
    """
    Test if the energy envelope of a frequency band is modulated by the 11-year cycle (Fit SSN).
    """
    
    # Update column names to English format
    target_residual_column = f'Residual_{model_name}'
    planetary_fit_column = f'Fit_SSN_{model_name}'

    print(f"  Testing band: {band_name} ({freq_band_days[0]}-{freq_band_days[1]} days)")

    # 1. Prepare data
    # Note: Changed '日期' to 'Date'
    data = data_full[['Date', target_residual_column, planetary_fit_column]].copy()
    
    # Filter clean data for analysis (both Fit SSN and Residual must exist)
    data_clean = data.dropna(subset=[target_residual_column, planetary_fit_column])
    if len(data_clean) < 5000: # Need enough data for filtering
        print(f"  Warning: Not enough clean data for {model_name} ({len(data_clean)} rows).")
        return None

    # --- Signal Processing ---
    
    # 2. Y (Residual) -> Bandpass Filter
    Y_filtered = bandpass_filter(
        data_clean[target_residual_column].values, 
        freq_band_days[0], 
        freq_band_days[1]
    )
    
    # 3. Y_filtered -> Energy Envelope
    Y_envelope = get_energy_envelope(
        Y_filtered, 
        smoothing_window_days=envelope_smoothing_days
    )
    
    data_clean.loc[:, 'Y_Envelope'] = Y_envelope
    
    # 4. X (Driver) -> Also smoothed
    #    We compare two slow variables: slow change of envelope vs slow change of 11-year cycle
    data_clean.loc[:, 'X_Driver_Smooth'] = data_clean[planetary_fit_column].rolling(
        window=envelope_smoothing_days, 
        center=True, 
        min_periods=int(envelope_smoothing_days * 0.1)
    ).mean()

    # --- Build Test Model ---
    
    # 5. Clean again (smoothing and filtering create new NaNs)
    data_model = data_clean.dropna(subset=['Y_Envelope', 'X_Driver_Smooth'])
    
    # 6. Define Train/Test masks
    # Note: Changed '日期' to 'Date'
    train_mask = (data_model['Date'] <= train_end_date)
    test_mask = (data_model['Date'] > train_end_date) & (data_model['Date'] <= test_end_date)

    X_train = data_model.loc[train_mask, ['X_Driver_Smooth']]
    Y_train = data_model.loc[train_mask, 'Y_Envelope']
    X_test = data_model.loc[test_mask, ['X_Driver_Smooth']]
    Y_test = data_model.loc[test_mask, 'Y_Envelope']

    if len(X_train) == 0 or len(X_test) == 0:
        print(f"  Error: Train or Test set is empty for band {band_name}.")
        return None

    # 7. Fit a simple LGBM: Y_Envelope = f(X_Driver_Smooth)
    #    We use it to capture potential non-linear relationships
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Use a very simple, anti-overfitting LGBM
    lgbm_tester = lgb.LGBMRegressor(
        n_estimators=50,
        num_leaves=5,
        learning_rate=0.05,
        random_state=42,
        n_jobs=1
    )
    
    lgbm_tester.fit(X_train_scaled, Y_train)
    
    # 8. Evaluate R^2 on the test set
    Y_pred = lgbm_tester.predict(X_test_scaled)
    r2 = r2_score(Y_test, Y_pred)
    
    print(f"  > {band_name} Test R2: {r2:.4f}")

    return {
        'model': model_name,
        'band_name': band_name,
        'low_days': freq_band_days[0],
        'high_days': freq_band_days[1],
        'r2_score': r2
    }


# === Main Execution Script ===
if __name__ == "__main__":
  
    # --- 1. Define bands to filter (Unit: Days) ---
    #    We filter short-term and mid-term frequencies
    BANDS_TO_TEST = {
        # Solar Rotation (Wide)
        'Rotation_Wide': [20, 40],
        # Solar Rotation (Core)
        'Rotation_Core': [25, 30],
        # Rieger Period
        'Rieger': [140, 170],
        # Annual Period
        'Annual': [350, 380],
        # QBO (Quasi-Biennial Oscillation)
        'QBO_1.5_3.0y': [1.5 * 365.25, 3.0 * 365.25],
        # QBO (Narrow Band)
        'QBO_2.2y': [2.0 * 365.25, 2.4 * 365.25]
        # Note: The 22y Hale cycle is too long, not suitable for energy envelope testing (it is the driver scale itself)
    }

    # --- 2. Define model split points ---
    model_splits = {
        'M8+2': '1996-08-01', 
        'M8+3': '1986-08-01',
        'M0+3': '1986-09-01', 
        'M0+2': '1996-08-01'
    }
    
    test_end_date = '2019-11-30'
    
    # --- 3. Load Data ---
    try:
        data_source_file ='../../results/05_p_m_a_model/p_model_4/residual/Summary_Fit_Results.csv' 
        # Columns: ['Date', 'Raw_SSN', 'Smoothed_SSN', 'SIDC_SSN', 'Fit_SSN_M8+2', 'Residual_M8+2', ...]
        print(f"Loading source data: {data_source_file}")
        # Note: Changed '日期' to 'Date'
        full_data = pd.read_csv(data_source_file, parse_dates=['Date'])
    except Exception as e:
        print(f"Fatal Error: Cannot read {data_source_file}. Error: {e}")
        exit()

    
    # --- 4. Start Screening ---
    all_results = []
    
    print("\n=== Start Screening for 'Energy Envelope Modulation' of Physical Bands ===")
    
    for model_name, split_date in model_splits.items():
        print(f"\n{'='*50}")
        print(f"Processing Model: {model_name} (Train End: {split_date})")
        print(f"{'='*50}")

        # Check if required columns exist (Using English names now)
        target_col = f'Residual_{model_name}'
        driver_col = f'Fit_SSN_{model_name}'
        if target_col not in full_data.columns or driver_col not in full_data.columns:
            print(f"Warning: Missing {target_col} or {driver_col}. Skipping this model.")
            continue

        for band_name, freq_band_days in BANDS_TO_TEST.items():
            result = run_modulation_test(
                data_full=full_data,
                model_name=model_name,
                train_end_date=split_date,
                test_end_date=test_end_date,
                band_name=band_name,
                freq_band_days=freq_band_days,
                envelope_smoothing_days=365 # Use 1-year smoothing window
            )
            if result:
                all_results.append(result)

    # --- 5. Summary Report ---
    if not all_results:
        print("\nAll analyses failed.")
        exit()

    print("\n\n" + "="*60)
    print("      Energy Envelope Modulation Screening - Final Summary Report")
    print("      (Testing Y_Envelope = f(X_Planetary_Fit) R2 score on Test Set)")
    print("="*60)

    results_df = pd.DataFrame(all_results)
    
    # Calculate average R2 for each band across 4 models
    summary = results_df.pivot_table(
        index='band_name', 
        columns='model', 
        values='r2_score'
    )
    
    # Calculate mean R2 and sort
    summary['Average_R2'] = summary.mean(axis=1)
    summary = summary.sort_values(by='Average_R2', ascending=False)
    
    print(summary.to_string(float_format="%.4f"))

    print("\n" + "="*60)
    print("Conclusion:")
    top_band = summary.index[0]
    top_r2 = summary.iloc[0]['Average_R2']
    
    if top_r2 > 0.05:
        print(f"✅ Strong Evidence: Band '{top_band}' (Avg R2={top_r2:.4f}) energy envelope")
        print("   has a strong modulation relationship with the 11-year planetary cycle.")
        print(f"\n👉 Recommendation: In the LGBM full frequency comb model, focus on modulation features of the {top_band} band.")
    elif top_r2 > 0.01:
        print(f"✅ Valid Evidence: Band '{top_band}' (Avg R2={top_r2:.4f}) shows a weak but stable modulation relationship.")
    else:
        print("❌ Insufficient Evidence: Correlation (R2) between energy envelopes of all bands and the 11-year cycle is very low.")
        print("   'Mod_' modulation features in LGBM might have limited effect or depend on other mechanisms.")

LightGBM imported successfully.
Scipy imported successfully (version: 1.16.3).
Loading source data: ../../results/05_p_m_a_model/p_model_4/residual/Summary_Fit_Results.csv

=== Start Screening for 'Energy Envelope Modulation' of Physical Bands ===

Processing Model: M8+2 (Train End: 1996-08-01)
  Testing band: Rotation_Wide (20-40 days)
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000163 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 53904, number of used features: 1
[LightGBM] [Info] Start training from score 29.679758
  > Rotation_Wide Test R2: 0.3943
  Testing band: Rotation_Core (25-30 days)
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000145 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is 

# Energy envelope filtering, initial fitting results are unsatisfactory.

In [2]:
import pandas as pd
import numpy as np
import time
import warnings
import os
import joblib

from sklearn.metrics import r2_score
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from skopt import BayesSearchCV
from skopt.space import Real, Integer

try:
    import lightgbm as lgb
    print("LightGBM imported successfully.")
except ImportError:
    print("Error: lightgbm library not found.")
    exit()

try:
    from astropy.time import Time
    import astropy.units as u
    from sunpy.coordinates import sun
except ImportError:
    print("Missing astronomy libraries.")
    exit()

warnings.filterwarnings('ignore')

def run_lgbm_optimized_comb_validation( # Renamed function
    data_full, model_name, train_start_date, train_end_date, test_end_date, model_save_dir
):
    """
    Run LGBM (Optimized)
    - Features: B0 + All basic cycles
    - Modulation: Only modulate QBO and Rotation (based on energy envelope screening)
    - Y Transform: QuantileTransformer
    - Tuning: Strong regularization
    - Functionality: Save model and full-cycle predictions
    """
    
    data = data_full.copy() 
    # Update column names to English
    target_residual_column = f'Residual_{model_name}'
    planetary_fit_column = f'Fit_SSN_{model_name}'
    
    # Create directory for saving models
    os.makedirs(model_save_dir, exist_ok=True) 
    
    print(f"\n=== LGBM Optimized (QBO+Rotation Modulation): {model_name} ===")
    start_time = time.time()
    
    if target_residual_column not in data.columns or planetary_fit_column not in data.columns:
        print(f"Error: Missing column {target_residual_column} or {planetary_fit_column}")
        return None, None

    # --- 1. Feature Engineering: B0 + Full Frequency Comb (All dates) ---
    print("1. Generating B0 + Full basic frequency comb features (Full cycle)...")
    times_astropy = Time(data['Date'].values)
    
    original_feature_names = []
    
    # B0 (Solar B0 angle)
    b0_deg = sun.B0(times_astropy).deg
    data['B0'] = b0_deg
    data['B0_sq'] = b0_deg ** 2
    original_feature_names.extend(['B0', 'B0_sq'])
    
    days_since_start = (data['Date'] - data['Date'].min()).dt.days
    
    # 1. Hale (22y)
    hale_rad = 2 * np.pi * days_since_start / (22.0 * 365.25)
    data['Hale_22y_sin'] = np.sin(hale_rad)
    data['Hale_22y_cos'] = np.cos(hale_rad)
    original_feature_names.extend(['Hale_22y_sin', 'Hale_22y_cos'])

    # 2. QBO (1.5y - 3y)
    qbo_periods = [1.7, 2.2, 2.7] 
    for p in qbo_periods:
        rad = 2 * np.pi * days_since_start / (p * 365.25)
        name = f'QBO_{p:.1f}y'
        data[f'{name}_sin'] = np.sin(rad)
        data[f'{name}_cos'] = np.cos(rad)
        original_feature_names.extend([f'{name}_sin', f'{name}_cos'])

    # 3. Annual (1y)
    annual_periods = [360, 365.25, 370] 
    for p in annual_periods:
        rad = 2 * np.pi * days_since_start / p
        name = f'Annual_{p:.0f}d'
        data[f'{name}_sin'] = np.sin(rad)
        data[f'{name}_cos'] = np.cos(rad)
        original_feature_names.extend([f'{name}_sin', f'{name}_cos'])

    # 4. Rieger (154d)
    rieger_rad = 2 * np.pi * days_since_start / 154
    data['Rieger_154d_sin'] = np.sin(rieger_rad)
    data['Rieger_154d_cos'] = np.cos(rieger_rad)
    original_feature_names.extend(['Rieger_154d_sin', 'Rieger_154d_cos'])

    # 5. Rotation (25d-30d)
    rotation_periods = [25, 26, 27, 28, 29, 30] 
    for p in rotation_periods:
        rad = 2 * np.pi * days_since_start / p
        name = f'Rotation_{p:.0f}d'
        data[f'{name}_sin'] = np.sin(rad)
        data[f'{name}_cos'] = np.cos(rad)
        original_feature_names.extend([f'{name}_sin', f'{name}_cos'])
    
    print(f"Basic features: {len(original_feature_names)}")

    # --- 2. Modulation Feature Engineering (Refactored) ---
    print("2. Generating modulation features (Full cycle)...")
    
    train_mask_dates = (data['Date'] >= '1855-12-02') & (data['Date'] <= train_end_date)
    train_mask_clean = train_mask_dates & data[target_residual_column].notna() & data[planetary_fit_column].notna()
    
    if train_mask_clean.sum() == 0:
        print(f"Error: No clean training data between {train_start_date} and {train_end_date}.")
        return None, None
        
    planetary_fit_values_train = data.loc[train_mask_clean, [planetary_fit_column]]
    scaler_fit = StandardScaler()
    scaler_fit.fit(planetary_fit_values_train)
    
    planetary_fit_values_full = data[[planetary_fit_column]].fillna(0) 
    data['Planetary_Scaled'] = scaler_fit.transform(planetary_fit_values_full).flatten()
    
    # --- Key Modification: Based on energy envelope screening results ---
    #    Only create modulation features for Rotation and QBO
    
    # Define list of basic feature prefixes allowed to be modulated
    modulation_allow_list = ['QBO_', 'Rotation_']
    
    features_to_modulate = []
    for feat_name in original_feature_names:
        # Check if feature name starts with 'QBO_' or 'Rotation_'
        if any(feat_name.startswith(prefix) for prefix in modulation_allow_list):
            features_to_modulate.append(feat_name)

    print(f"Selected {len(features_to_modulate)} features for modulation (based on energy envelope test).")
    
    # Create modulation features (only for allowed list)
    modulation_feature_names = []
    for feature in features_to_modulate: # <--- Loop only through filtered subset
        new_feature_name = f'Mod_{feature}'
        data.loc[:, new_feature_name] = data['Planetary_Scaled'] * data[feature]
        modulation_feature_names.append(new_feature_name)
    
    # --- End of Modification ---
    
    feature_names = original_feature_names + modulation_feature_names
    print(f"Total features: {len(feature_names)} (30 Basic + {len(modulation_feature_names)} Modulated)")
    
    # --- 3. Data Split and Transformation (For Training) ---
    data_clean = data.dropna(subset=[target_residual_column] + feature_names)
    
    X = data_clean[feature_names]
    Y_raw = data_clean[target_residual_column]
    groups = data_clean['Date'].dt.year
    dates = data_clean['Date']
    
    train_mask = (dates >= train_start_date) & (dates <= train_end_date)
    test_start_date_dt = pd.to_datetime(train_end_date) + pd.Timedelta(days=1)
    test_mask = (dates >= test_start_date_dt) & (dates <= test_end_date)
    
    X_train_raw = X[train_mask]
    Y_train_raw = Y_raw[train_mask]
    X_test_raw = X[test_mask]
    Y_test_raw = Y_raw[test_mask]
    groups_train = groups[train_mask]
    
    if len(X_train_raw) == 0 or len(X_test_raw) == 0:
        print("Error: Train or Test set is empty")
        return None, None

    # a. Fit X Transformer
    feature_scaler = StandardScaler()
    X_train = feature_scaler.fit_transform(X_train_raw)
    X_test = feature_scaler.transform(X_test_raw)

    # b. Fit Y Transformer
    print("Applying QuantileTransformer to stabilize target variable Y (Residual)...")
    target_transformer = QuantileTransformer(output_distribution='normal', random_state=42)
    Y_train = target_transformer.fit_transform(Y_train_raw.values.reshape(-1, 1)).flatten()
    Y_test = target_transformer.transform(Y_test_raw.values.reshape(-1, 1)).flatten()
    
    print(f"Data: Train {len(X_train)}, Test {len(X_test)}")
    
    # --- 4. Bayesian Optimization - LGBM (Strong Regularization) ---
    n_splits = max(2, min(5, groups_train.nunique()))
    gkf = GroupKFold(n_splits=n_splits)
    
    search_spaces = {
        'n_estimators': Integer(100, 800),
        'num_leaves': Integer(5, 30),
        'learning_rate': Real(1e-3, 0.1, 'log-uniform'),
        'reg_alpha': Real(0.1, 20.0, 'log-uniform'), 
        'reg_lambda': Real(0.1, 20.0, 'log-uniform'),
        'subsample': Real(0.7, 1.0, 'uniform'),
        'colsample_bytree': Real(0.7, 1.0, 'uniform')
    }
    
    lgb_model = lgb.LGBMRegressor(random_state=42, n_jobs=1, 
                                  objective='regression_l1')

    opt_lgbm = BayesSearchCV(
        lgb_model, 
        search_spaces, 
        n_iter=30, 
        cv=gkf, 
        n_jobs=-1,
        random_state=42, 
        scoring='r2'
    )
    
    print("Training LGBM (Optimized)...")
    opt_lgbm.fit(X_train, Y_train, groups=groups_train)
    
    
    # --- 5. Result Analysis ---
    best_model = opt_lgbm.best_estimator_
    Y_pred_test = best_model.predict(X_test)
    final_r2 = r2_score(Y_test, Y_pred_test) 
    
    print(f"\nLGBM Optimized R2 (on transformed Y): {final_r2:.4f}")
    if final_r2 > 0:
        print("✅ Success! Model R2 is positive.")
    else:
        print("❌ R2 is still negative.")
        
    print(f"Best Params: {opt_lgbm.best_params_}")
    
    importances = pd.Series(best_model.feature_importances_, index=feature_names)
    print("\nFeature Importance (Top 20):")
    top_features = importances.sort_values(ascending=False).head(20) 
    print(top_features.to_string())
    
    mod_features_in_top = [f for f in top_features.index if f.startswith('Mod_')]
    print(f"\nIn Top 20, {len(mod_features_in_top)} are modulation features.")
    
    # --- 6. New Functionality: Save Model ---
    print("\nSaving model and Scalers...")
    # Use the model_save_dir variable
    model_path = os.path.join(model_save_dir, f"{model_name}_best_lgbm.joblib")
    fscaler_path = os.path.join(model_save_dir, f"{model_name}_feature_scaler.joblib")
    tscaler_path = os.path.join(model_save_dir, f"{model_name}_target_transformer.joblib")
    
    joblib.dump(best_model, model_path)
    joblib.dump(feature_scaler, fscaler_path)
    joblib.dump(target_transformer, tscaler_path)
    print(f"Model saved: {model_path}")

    # --- 7. New Functionality: Generate Full Cycle Predictions ---
    print("Generating full cycle (to 2050) predictions...")
    X_full = data[feature_names].fillna(0) 
    
    X_full_scaled = feature_scaler.transform(X_full)
    Y_pred_full_transformed = best_model.predict(X_full_scaled)
    
    Y_pred_full_raw = target_transformer.inverse_transform(Y_pred_full_transformed.reshape(-1, 1)).flatten()
    
    pred_col_name = f'LGBM_Pred_Opt_{model_name}' # Optimized column name
    resid_col_name = f'LGBM_Resid_Opt_{model_name}' # Optimized column name
    
    data[pred_col_name] = Y_pred_full_raw
    data[resid_col_name] = data[target_residual_column] - data[pred_col_name]
    
    stats_dict = {
        'model': model_name,
        'r2_score_transformed': final_r2,
        'n_features': len(feature_names),
        'n_mod_features': len(modulation_feature_names),
        'top_20_mod_count': len(mod_features_in_top),
        'best_params': opt_lgbm.best_params_,
        'training_time': time.time() - start_time
    }

    return data[['Date', pred_col_name, resid_col_name]], stats_dict


# Main Execution
if __name__ == "__main__":
    model_splits = {
        'M8+2': '1996-08-01', 
        'M8+3': '1986-08-01',
        'M0+3': '1986-09-01', 
        'M0+2': '1996-08-01'
    }
    
    # Define directory for joblib files
    # You can change this path if needed
    MODEL_SAVE_DIR = "../../results/05_p_m_a_model/m_model_4/LGBM_Models_Optimized" 

    print("=== Run LGBM (Physically Screened Optimized Version) ===")
    print("Strategy: Basic full frequency comb + (QBO+Rotation) Modulation + Target Stabilization + Strong Regularization")
    
    try:
        # Note: Updated to match Code 2's input path
        data_source_file = "../../results/05_p_m_a_model/p_model_4/residual/Summary_Fit_Results.csv"
        print(f"\nLoading source data: {data_source_file}")
        full_data_to_save = pd.read_csv(data_source_file, parse_dates=['Date'])
    except Exception as e:
        print(f"Fatal Error: Cannot read {data_source_file}. Error: {e}")
        exit()

    max_date_in_file = full_data_to_save['Date'].max()
    target_date = pd.to_datetime('2050-12-31')
    
    if max_date_in_file < target_date:
        print(f"Warning: Source data max date is {max_date_in_file}.")
        print(f"Warning: Extending date range to {target_date}.")
        all_dates_df = pd.DataFrame({'Date': pd.date_range(
            start=full_data_to_save['Date'].min(), 
            end=target_date, 
            freq='D'
        )})
        full_data_to_save = pd.merge(all_dates_df, full_data_to_save, on='Date', how='left')

    results_stats_list = []
    
    for model_name, split_date in model_splits.items():
        print(f"\n{'='*50}")
        print(f"Processing Model: {model_name} (Train End: {split_date})")
        print(f"{'='*50}")
        
        result_df, stats_dict = run_lgbm_optimized_comb_validation(
            data_full=full_data_to_save.copy(), 
            model_name=model_name,
            train_start_date='1855-12-02',
            train_end_date=split_date,
            test_end_date='2019-11-30',
            model_save_dir=MODEL_SAVE_DIR
        )
        
        if result_df is not None and stats_dict is not None:
            results_stats_list.append(stats_dict)
            full_data_to_save = pd.merge(full_data_to_save, result_df, on='Date', how='left')
    
    if results_stats_list:
        # Output filename as provided in the prompt
        output_filename = "../../results/05_p_m_a_model/m_model_4/LGBM_Optimized.csv" 
        print(f"\n{'='*60}")
        print(f"Saving optimized quadratic fit and residuals for all models to: {output_filename}")
        try:
            full_data_to_save.to_csv(output_filename, index=False, float_format='%.6f')
            print(f"Save successful. File contains {len(full_data_to_save)} rows.")
        except Exception as e:
            print(f"Failed to save CSV: {e}")
    
        results_to_print = []
        for r in results_stats_list:
            results_to_print.append({
                'model': r['model'],
                'r2_score': r['r2_score_transformed'],
                'n_feat_total': r['n_features'],
                'n_feat_mod': r['n_mod_features'],
                'top_20_mod': r['top_20_mod_count'],
                'time(s)': r['training_time']
            })
            
        results_df = pd.DataFrame(results_to_print)
        print("\n" + "="*60)
        print("LGBM Physically Screened Optimized Version Summary")
        print("="*60)
        print(results_df.to_string(index=False, float_format="%.4f"))
        
        avg_r2 = results_df['r2_score'].mean()
        print(f"\nAverage R2 (on transformed Y): {avg_r2:.4f}")
        
        # Compare with previous baseline 0.1265 (or 0.1198)
        if avg_r2 > 0.13:
            print(f"✅✅ Huge Success: Optimized R2 ({avg_r2:.4f}) is higher than baseline (approx 0.12)!")
        elif avg_r2 > 0.11:
            print(f"✅ Success: Optimized R2 ({avg_r2:.4f}) is comparable to baseline (approx 0.12).")
            print("   This proves the model robustness, and it has stronger physical meaning.")
        else:
            print(f"❌ Note: Optimized R2 ({avg_r2:.4f}) is lower than baseline (approx 0.12).")
            print("   This might mean there are still weak signals in the deleted features.")

    else:
        print("\nAll models failed.")

LightGBM imported successfully.
=== Run LGBM (Physically Screened Optimized Version) ===
Strategy: Basic full frequency comb + (QBO+Rotation) Modulation + Target Stabilization + Strong Regularization

Loading source data: ../../results/05_p_m_a_model/p_model_4/residual/Summary_Fit_Results.csv

Processing Model: M8+2 (Train End: 1996-08-01)

=== LGBM Optimized (QBO+Rotation Modulation): M8+2 ===
1. Generating B0 + Full basic frequency comb features (Full cycle)...
Basic features: 30
2. Generating modulation features (Full cycle)...
Selected 18 features for modulation (based on energy envelope test).
Total features: 48 (30 Basic + 18 Modulated)
Applying QuantileTransformer to stabilize target variable Y (Residual)...
Data: Train 51378, Test 8521
Training LGBM (Optimized)...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008760 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12238
[LightGBM] [Info] Nu

# Supplementing long-term information that may be missed by planetary fitting methods

In [3]:
import pandas as pd
import numpy as np
import time
import warnings
import os
import joblib

from sklearn.metrics import r2_score
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from skopt import BayesSearchCV
from skopt.space import Real, Integer

# --- Core: Non-linear Model Import ---
try:
    import lightgbm as lgb
    print("LightGBM imported successfully.")
except ImportError:
    print("Error: lightgbm library not found.")
    print("Please install: pip install lightgbm")
    exit()

# Astronomy Library Import
try:
    from astropy.time import Time
    import astropy.units as u
    from sunpy.coordinates import sun
except ImportError:
    print("Missing astronomy libraries.")
    exit()

warnings.filterwarnings('ignore')

def run_lgbm_full_comb_validation(
    data_full, model_name, train_start_date, train_end_date, 
    test_end_date, model_save_dir
):
    """
    Run LGBM (Final Full Frequency Comb Version)
    - Features: B0 + All famous cycles (22y, QBO, 1y, 154d, 25-30d)
    - Modulation: 11-year planetary fit
    - Y Transform: QuantileTransformer
    - Tuning: Strong regularization
    - New: Save model and full-cycle predictions
    """
    
    # Use the passed complete data
    data = data_full.copy() 
    
    # Update column names to English
    target_residual_column = f'Residual_{model_name}'
    planetary_fit_column = f'Fit_SSN_{model_name}'
    
    # Ensure model save directory exists
    os.makedirs(model_save_dir, exist_ok=True)
   
    print(f"\n=== LGBM Final Full Frequency Comb Version: {model_name} ===")
    start_time = time.time()
    
    # Check if necessary columns exist
    if target_residual_column not in data.columns or planetary_fit_column not in data.columns:
        print(f"Error: Missing column {target_residual_column} or {planetary_fit_column}")
        return None, None

    # --- 1. Feature Engineering: B0 + Full Frequency Comb (All dates) ---
    print("1. Generating B0 Ephemeris Features + Full Frequency Comb Features (Full Cycle)...")
    times_astropy = Time(data['Date'].values)
    
    original_feature_names = []
    
    # B0 (Heliographic latitude) - Real annual amplitude
    b0_deg = sun.B0(times_astropy).deg
    data['B0'] = b0_deg
    data['B0_sq'] = b0_deg ** 2
    original_feature_names.extend(['B0', 'B0_sq'])
    
    # Time-based fixed frequency features
    days_since_start = (data['Date'] - data['Date'].min()).dt.days
    
    # === Define our Frequency Comb ===
    
    # 1. Hale (22y)
    hale_rad = 2 * np.pi * days_since_start / (22.0 * 365.25)
    data['Hale_22y_sin'] = np.sin(hale_rad)
    data['Hale_22y_cos'] = np.cos(hale_rad)
    original_feature_names.extend(['Hale_22y_sin', 'Hale_22y_cos'])

    # 2. QBO (1.5y - 3y)
    qbo_periods = [1.7, 2.2, 2.7] # Years
    for p in qbo_periods:
        rad = 2 * np.pi * days_since_start / (p * 365.25)
        name = f'QBO_{p:.1f}y'
        data[f'{name}_sin'] = np.sin(rad)
        data[f'{name}_cos'] = np.cos(rad)
        original_feature_names.extend([f'{name}_sin', f'{name}_cos'])

    # 3. Annual (1y)
    annual_periods = [360, 365.25, 370] # Days
    for p in annual_periods:
        rad = 2 * np.pi * days_since_start / p
        name = f'Annual_{p:.0f}d'
        data[f'{name}_sin'] = np.sin(rad)
        data[f'{name}_cos'] = np.cos(rad)
        original_feature_names.extend([f'{name}_sin', f'{name}_cos'])

    # 4. Rieger (154d)
    rieger_rad = 2 * np.pi * days_since_start / 154
    data['Rieger_154d_sin'] = np.sin(rieger_rad)
    data['Rieger_154d_cos'] = np.cos(rieger_rad)
    original_feature_names.extend(['Rieger_154d_sin', 'Rieger_154d_cos'])

    # 5. Rotation (25d-30d)
    rotation_periods = [25, 26, 27, 28, 29, 30] # Days
    for p in rotation_periods:
        rad = 2 * np.pi * days_since_start / p
        name = f'Rotation_{p:.0f}d'
        data[f'{name}_sin'] = np.sin(rad)
        data[f'{name}_cos'] = np.cos(rad)
        original_feature_names.extend([f'{name}_sin', f'{name}_cos'])
    
    
    # --- 2. Modulation Feature Engineering (Refactored) ---
    # Goal: Fit Scaler on training set, Transform on full cycle
    print("2. Generating modulation features (Full Cycle)...")
    
    # a. Create training mask for fitting Scaler
    # Training set mask (only for fitting Scaler and Model)
    train_mask_dates = (data['Date'] >= train_start_date) & (data['Date'] <= train_end_date)
    # Must be clean, non-NaN data
    train_mask_clean = train_mask_dates & data[target_residual_column].notna() & data[planetary_fit_column].notna()
    
    if train_mask_clean.sum() == 0:
        print(f"Error: No clean training data between {train_start_date} and {train_end_date}.")
        return None, None
        
    # b. Fit Modulator (11-year cycle signal) Scaler (Only on training data)
    planetary_fit_values_train = data.loc[train_mask_clean, [planetary_fit_column]]
    scaler_fit = StandardScaler()
    scaler_fit.fit(planetary_fit_values_train)
    
    # c. Transform (Full Cycle)
    # Warning: If planetary_fit_column is NaN in the future, this will create NaNs.
    # We fill NaNs with 0 (assuming future fit is 0 if missing, not ideal but prevents crash)
    planetary_fit_values_full = data[[planetary_fit_column]].fillna(0) 
    data['Planetary_Scaled'] = scaler_fit.transform(planetary_fit_values_full).flatten()

    # --- [ *** New Code *** ] ---
    # 2.c.1: Create rate of change feature (1st order difference)
    # This represents the "Rising Phase" (positive) or "Declining Phase" (negative) of the 11-year cycle
    print("... Adding 'Planetary_Scaled_Diff' (Rate of Change) feature...")
    data['Planetary_Scaled_Diff'] = data['Planetary_Scaled'].diff(1).fillna(0)
    
    # 2.c.2: Add it to the *Basic* feature list
    # Key: This way it will automatically get its own "Mod_" version in the next step
    original_feature_names.append('Planetary_Scaled_Diff')
    # --- [ *** End of New Code *** ] ---
    
    # d. Create Modulation Features (Full Cycle)
    modulation_feature_names = []
    for feature in original_feature_names:
        new_feature_name = f'Mod_{feature}'
        data.loc[:, new_feature_name] = data['Planetary_Scaled'] * data[feature]
        modulation_feature_names.append(new_feature_name)
    
    feature_names = original_feature_names + modulation_feature_names
    print(f"Total features: {len(feature_names)} (Full Frequency Comb + Rate of Change)")
    
    # --- 3. Data Split and Transformation (For Training) ---
    
    # Only use rows with valid residuals for training and testing
    data_clean = data.dropna(subset=[target_residual_column] + feature_names)
    
    X = data_clean[feature_names]
    Y_raw = data_clean[target_residual_column] # Y Raw values
    groups = data_clean['Date'].dt.year
    dates = data_clean['Date']
    
    train_mask = (dates >= train_start_date) & (dates <= train_end_date)
    test_start_date_dt = pd.to_datetime(train_end_date) + pd.Timedelta(days=1)
    test_mask = (dates >= test_start_date_dt) & (dates <= test_end_date)
    
    X_train_raw = X[train_mask]
    Y_train_raw = Y_raw[train_mask]
    X_test_raw = X[test_mask]
    Y_test_raw = Y_raw[test_mask]
    groups_train = groups[train_mask]
    
    if len(X_train_raw) == 0 or len(X_test_raw) == 0:
        print("Error: Train or Test set is empty")
        return None, None

    # a. Fit X Transformer (StandardScaler)
    feature_scaler = StandardScaler()
    X_train = feature_scaler.fit_transform(X_train_raw)
    X_test = feature_scaler.transform(X_test_raw)

    # b. Fit Y Transformer (QuantileTransformer) - Key Fix
    print("Applying QuantileTransformer to stabilize target variable Y (Residual)...")
    target_transformer = QuantileTransformer(output_distribution='normal', random_state=42)
    Y_train = target_transformer.fit_transform(Y_train_raw.values.reshape(-1, 1)).flatten()
    Y_test = target_transformer.transform(Y_test_raw.values.reshape(-1, 1)).flatten()
    
    print(f"Data: Train {len(X_train)}, Test {len(X_test)}")
    
    # --- 4. Bayesian Optimization - LGBM (Strong Regularization) ---
    n_splits = max(2, min(5, groups_train.nunique()))
    gkf = GroupKFold(n_splits=n_splits)
    
    search_spaces = {
        'n_estimators': Integer(100, 800),
        'num_leaves': Integer(5, 30),
        'learning_rate': Real(1e-3, 0.1, 'log-uniform'),
        'reg_alpha': Real(0.1, 20.0, 'log-uniform'), 
        'reg_lambda': Real(0.1, 20.0, 'log-uniform'),
        'subsample': Real(0.7, 1.0, 'uniform'),
        'colsample_bytree': Real(0.7, 1.0, 'uniform')
    }
    
    lgb_model = lgb.LGBMRegressor(
        random_state=42, 
        n_jobs=1, 
        objective='regression_l1',
        force_row_wise=True, # Force row-wise to eliminate auto-detection warnings
        verbose=-1           # Suppress all LightGBM internal output (recommended)
    )

    opt_lgbm = BayesSearchCV(
        lgb_model, 
        search_spaces, 
        n_iter=100, # <-- Increased from 30 to 50 (or 100 as per code)
        cv=gkf, 
        n_jobs=-2,
        random_state=42, 
        scoring='r2'
    )
    
    print("Training LGBM (Full Frequency Comb Version)...")
    opt_lgbm.fit(X_train, Y_train, groups=groups_train)
    
    # --- 5. Result Analysis ---
    best_model = opt_lgbm.best_estimator_
    Y_pred_test = best_model.predict(X_test)
    final_r2 = r2_score(Y_test, Y_pred_test) 
    
    print(f"\nLGBM Full Comb Version R2 (on transformed Y): {final_r2:.4f}")
    if final_r2 > 0:
        print("✅ Success! Model R2 is positive.")
    else:
        print("❌ R2 is still negative.")
        
    print(f"Best Params: {opt_lgbm.best_params_}")
    
    # Feature Importance
    importances = pd.Series(best_model.feature_importances_, index=feature_names)
    print("\nFeature Importance (Top 20):")
    top_features = importances.sort_values(ascending=False).head(20) 
    print(top_features.to_string())
    
    mod_features_in_top = [f for f in top_features.index if f.startswith('Mod_')]
    print(f"\nIn Top 20, {len(mod_features_in_top)} are modulation features.")
    
    # --- 6. New Functionality: Save Model ---
    print("\nSaving model and Scalers...")
    
    # Updated to use model_save_dir
    model_path = os.path.join(model_save_dir, f"{model_name}_best_lgbm.joblib")
    fscaler_path = os.path.join(model_save_dir, f"{model_name}_feature_scaler.joblib")
    tscaler_path = os.path.join(model_save_dir, f"{model_name}_target_transformer.joblib")
    
    joblib.dump(best_model, model_path)
    joblib.dump(feature_scaler, fscaler_path)
    joblib.dump(target_transformer, tscaler_path)
    print(f"Model saved: {model_path}")

    # --- 7. New Functionality: Generate Full Cycle Predictions ---
    print("Generating full cycle (to 2050) predictions...")
    
    # Prepare X features for full cycle
    # Must fill NaNs in features (e.g., if Planetary_Scaled is NaN at the end)
    X_full = data[feature_names].fillna(0) 
    
    X_full_scaled = feature_scaler.transform(X_full)
    Y_pred_full_transformed = best_model.predict(X_full_scaled)
    
    # Inverse transform back to original scale
    Y_pred_full_raw = target_transformer.inverse_transform(Y_pred_full_transformed.reshape(-1, 1)).flatten()
    
    # Create prediction column and secondary residual column
    pred_col_name = f'LGBM_Pred_{model_name}'
    resid_col_name = f'LGBM_Resid_{model_name}'
    
    data[pred_col_name] = Y_pred_full_raw
    # Original Residual - LGBM Predicted Residual = Secondary Residual
    data[resid_col_name] = data[target_residual_column] - data[pred_col_name]
    
    # Prepare stats dictionary
    stats_dict = {
        'model': model_name,
        'r2_score_transformed': final_r2,
        'n_features': len(feature_names),
        'top_20_mod_count': len(mod_features_in_top),
        'best_params': opt_lgbm.best_params_,
        'training_time': time.time() - start_time
    }

    # Return DataFrame with new columns (only necessary columns) and stats results
    return data[['Date', pred_col_name, resid_col_name]], stats_dict


# Main Execution - Run LGBM Full Frequency Comb Version
if __name__ == "__main__":
    model_splits = {
        'M8+2': '1996-08-01', 
        'M8+3': '1986-08-01',
        'M0+3': '1986-09-01', 
        'M0+2': '1996-08-01'
    }
    
    print("=== Run LGBM (Full Frequency Comb Version: B0 + 22y, QBO, 1y, 154d, 25-30d) ===")
    print("Strategy: Physical Full Frequency Comb + 11-year Modulation + Target Stabilization + Strong Regularization")
    print("New: Save model and full cycle secondary residuals")

    MODEL_SAVE_DIR = "../../results/05_p_m_a_model/m_model_4/LGBM_Models1" 
    
    # --- New: Load and Prepare Data Outside Loop ---
    try:
        data_source_file = "../../results/05_p_m_a_model/p_model_4/residual/Summary_Fit_Results.csv"
        print(f"\nLoading source data: {data_source_file}")
        full_data_to_save = pd.read_csv(data_source_file, parse_dates=['Date'])
    except Exception as e:
        print(f"Fatal Error: Cannot read {data_source_file}. Error: {e}")
        exit()

    # Check and extend dates to 2050
    max_date_in_file = full_data_to_save['Date'].max()
    target_date = pd.to_datetime('2050-12-31')
    
    if max_date_in_file < target_date:
        print(f"Warning: Source data max date is {max_date_in_file}.")
        print(f"Warning: Extending date range to {target_date}.")
        print("Warning: Please ensure 'Fit_SSN' (Planetary Fit) column in source file is extrapolated to 2050, otherwise predictions will be inaccurate!")
        
        all_dates_df = pd.DataFrame({'Date': pd.date_range(
            start=full_data_to_save['Date'].min(), 
            end=target_date, 
            freq='D'
        )})
        full_data_to_save = pd.merge(all_dates_df, full_data_to_save, on='Date', how='left')

    # To save stats results
    results_stats_list = []
    
    # Loop through each model
    for model_name, split_date in model_splits.items():
        print(f"\n{'='*50}")
        print(f"Processing Model: {model_name} (Train End: {split_date})")
        print(f"{'='*50}")
        
        # Pass full data
        result_df, stats_dict = run_lgbm_full_comb_validation(
            data_full=full_data_to_save.copy(), # Pass a copy of data
            model_name=model_name,
            train_start_date='1855-12-02',
            train_end_date=split_date,
            test_end_date='2019-11-30',
            model_save_dir=MODEL_SAVE_DIR # Pass the directory explicitly
        )
        
        if result_df is not None and stats_dict is not None:
            results_stats_list.append(stats_dict)
            # Merge this model's results back to main DataFrame
            full_data_to_save = pd.merge(full_data_to_save, result_df, on='Date', how='left')
    
    # --- New: Save CSV containing all results ---
    if results_stats_list:
        output_filename = "../../results/05_p_m_a_model/m_model_4/LGBM_results1.csv"
        print(f"\n{'='*60}")
        print(f"Saving secondary fit and residuals for all models to: {output_filename}")
        try:
            full_data_to_save.to_csv(output_filename, index=False, float_format='%.6f')
            print(f"Save successful. File contains {len(full_data_to_save)} rows.")
        except Exception as e:
            print(f"Failed to save CSV: {e}")
        
        # --- Print Summary Report (Using stats data) ---
        results_to_print = []
        for r in results_stats_list:
            results_to_print.append({
                'model': r['model'],
                'r2_score': r['r2_score_transformed'],
                'n_features': r['n_features'],
                'top_20_mod': r['top_20_mod_count'],
                'time(s)': r['training_time']
            })
            
        results_df = pd.DataFrame(results_to_print)
        print("\n" + "="*60)
        print("LGBM Full Frequency Comb Version Summary")
        print("="*60)
        print(results_df.to_string(index=False, float_format="%.4f"))
        
        avg_r2 = results_df['r2_score'].mean()
        print(f"\nAverage R2 (on transformed Y): {avg_r2:.4f}")
        
        positive_r2 = results_df[results_df['r2_score'] > 0]
        if len(positive_r2) == 4:
            print("✅✅✅ Huge Success: All 4 models have positive R2!")
        elif len(positive_r2) > 0:
            print(f"✅ Success: {len(positive_r2)} models obtained positive R2.")
        else:
            print("❌ Failure: All models still have negative R2.")
        
        if results_df['top_20_mod'].mean() >= 10: 
             print("💡 Insight: Modulation features (Mod_) play a significant role in the models.")

    else:
        print("\nAll models failed.")

LightGBM imported successfully.
=== Run LGBM (Full Frequency Comb Version: B0 + 22y, QBO, 1y, 154d, 25-30d) ===
Strategy: Physical Full Frequency Comb + 11-year Modulation + Target Stabilization + Strong Regularization
New: Save model and full cycle secondary residuals

Loading source data: ../../results/05_p_m_a_model/p_model_4/residual/Summary_Fit_Results.csv

Processing Model: M8+2 (Train End: 1996-08-01)

=== LGBM Final Full Frequency Comb Version: M8+2 ===
1. Generating B0 Ephemeris Features + Full Frequency Comb Features (Full Cycle)...
2. Generating modulation features (Full Cycle)...
... Adding 'Planetary_Scaled_Diff' (Rate of Change) feature...
Total features: 62 (Full Frequency Comb + Rate of Change)
Applying QuantileTransformer to stabilize target variable Y (Residual)...
Data: Train 51378, Test 8521
Training LGBM (Full Frequency Comb Version)...

LGBM Full Comb Version R2 (on transformed Y): 0.0159
✅ Success! Model R2 is positive.
Best Params: OrderedDict({'colsample_bytree

# Based on the 11 rate of change parameters, add "acceleration." This is the optimal approach.

In [1]:
import pandas as pd
import numpy as np
import time
import warnings
import os
import joblib

from sklearn.metrics import r2_score
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from skopt import BayesSearchCV
from skopt.space import Real, Integer

# --- Core: Non-linear Model Import ---
try:
    import lightgbm as lgb
    print("LightGBM imported successfully.")
except ImportError:
    print("Error: lightgbm library not found.")
    print("Please install: pip install lightgbm")
    exit()

# Astronomy Library Import
try:
    from astropy.time import Time
    import astropy.units as u
    from sunpy.coordinates import sun
except ImportError:
    print("Missing astronomy libraries.")
    exit()

warnings.filterwarnings('ignore')

def run_lgbm_full_comb_validation(
    data_full, model_name, train_start_date, train_end_date, 
    test_end_date, model_save_dir
):
    """
    Run LGBM (Final Full Frequency Comb Version)
    - Features: B0 + All famous cycles (22y, QBO, 1y, 154d, 25-30d)
    - Modulation: 11-year planetary fit
    - Y Transform: QuantileTransformer
    - Tuning: Strong regularization
    - New: Save model and full-cycle predictions
    """
    
    # Use the passed complete data
    data = data_full.copy() 
    
    # Update column names to English
    target_residual_column = f'Residual_{model_name}'
    planetary_fit_column = f'Fit_SSN_{model_name}'
    
    # [cite_start]Ensure model save directory exists [cite: 87]
    os.makedirs(model_save_dir, exist_ok=True)
    
    print(f"\n=== LGBM Final Full Frequency Comb Version: {model_name} ===")
    start_time = time.time()
    
    # Check if necessary columns exist
    if target_residual_column not in data.columns or planetary_fit_column not in data.columns:
        print(f"Error: Missing column {target_residual_column} or {planetary_fit_column}")
        return None, None

    # --- 1. Feature Engineering: B0 + Full Frequency Comb (All dates) ---
    print("1. Generating B0 Ephemeris Features + Full Frequency Comb Features (Full Cycle)...")
    times_astropy = Time(data['Date'].values)
    
    original_feature_names = []
    
    # B0 (Heliographic latitude) - Real annual amplitude
    b0_deg = sun.B0(times_astropy).deg
    data['B0'] = b0_deg
    data['B0_sq'] = b0_deg ** 2
    original_feature_names.extend(['B0', 'B0_sq'])
    
    # Time-based fixed frequency features
    days_since_start = (data['Date'] - data['Date'].min()).dt.days
    
    # === Define our Frequency Comb ===
    
    # 1. Hale (22y)
    hale_rad = 2 * np.pi * days_since_start / (22.0 * 365.25)
    data['Hale_22y_sin'] = np.sin(hale_rad)
    data['Hale_22y_cos'] = np.cos(hale_rad)
    original_feature_names.extend(['Hale_22y_sin', 'Hale_22y_cos'])

    # 2. QBO (1.5y - 3y)
    qbo_periods = [1.7, 2.2, 2.7] # Years
    for p in qbo_periods:
        rad = 2 * np.pi * days_since_start / (p * 365.25)
        name = f'QBO_{p:.1f}y'
        data[f'{name}_sin'] = np.sin(rad)
        data[f'{name}_cos'] = np.cos(rad)
        original_feature_names.extend([f'{name}_sin', f'{name}_cos'])

    # 3. Annual (1y)
    annual_periods = [360, 365.25, 370] # Days
    for p in annual_periods:
        rad = 2 * np.pi * days_since_start / p
        name = f'Annual_{p:.0f}d'
        data[f'{name}_sin'] = np.sin(rad)
        data[f'{name}_cos'] = np.cos(rad)
        original_feature_names.extend([f'{name}_sin', f'{name}_cos'])

    # 4. Rieger (154d)
    rieger_rad = 2 * np.pi * days_since_start / 154
    data['Rieger_154d_sin'] = np.sin(rieger_rad)
    data['Rieger_154d_cos'] = np.cos(rieger_rad)
    original_feature_names.extend(['Rieger_154d_sin', 'Rieger_154d_cos'])

    # 5. Rotation (25d-30d)
    rotation_periods = [25, 26, 27, 28, 29, 30] # Days
    for p in rotation_periods:
        rad = 2 * np.pi * days_since_start / p
        name = f'Rotation_{p:.0f}d'
        data[f'{name}_sin'] = np.sin(rad)
        data[f'{name}_cos'] = np.cos(rad)
        original_feature_names.extend([f'{name}_sin', f'{name}_cos'])
    
    
    # --- 2. Modulation Feature Engineering (Refactored) ---
    # Goal: Fit Scaler on training set, Transform on full cycle
    print("2. Generating modulation features (Full Cycle)...")
    
    # a. Create training mask for fitting Scaler
    # Training set mask (only for fitting Scaler and Model)
    train_mask_dates = (data['Date'] >= train_start_date) & (data['Date'] <= train_end_date)
    # Must be clean, non-NaN data
    train_mask_clean = train_mask_dates & data[target_residual_column].notna() & data[planetary_fit_column].notna()
    
    if train_mask_clean.sum() == 0:
        print(f"Error: No clean training data between {train_start_date} and {train_end_date}.")
        return None, None
        
    # b. Fit Modulator (11-year cycle signal) Scaler (Only on training data)
    planetary_fit_values_train = data.loc[train_mask_clean, [planetary_fit_column]]
    scaler_fit = StandardScaler()
    scaler_fit.fit(planetary_fit_values_train)
    
    # c. Transform (Full Cycle)
    # Warning: If planetary_fit_column is NaN in the future, this will create NaNs.
    # We fill NaNs with 0 (assuming future fit is 0 if missing, not ideal but prevents crash)
    planetary_fit_values_full = data[[planetary_fit_column]].fillna(0) 
    data['Planetary_Scaled'] = scaler_fit.transform(planetary_fit_values_full).flatten()

    # --- [ *** Modified Here *** ] ---
    
    # 2.c.1: Create Rate of Change feature (Velocity)
    print("... Adding 'Planetary_Scaled_Diff' (Velocity) feature...")
    data['Planetary_Scaled_Diff'] = data['Planetary_Scaled'].diff(1).fillna(0)
    
    # 2.c.2: Create Acceleration feature (Rate of change of Velocity)
    print("... Adding 'Planetary_Scaled_Accel' (Acceleration) feature...")
    data['Planetary_Scaled_Accel'] = data['Planetary_Scaled_Diff'].diff(1).fillna(0)

    # 2.c.3: Add both to the *Basic* feature list
    # Key: This way they will automatically get their own "Mod_" versions in the next step
    original_feature_names.append('Planetary_Scaled_Diff')
    original_feature_names.append('Planetary_Scaled_Accel')
    # --- [ *** End of Modification *** ] ---
    
    # d. Create Modulation Features (Full Cycle)
    modulation_feature_names = []
    for feature in original_feature_names:
        new_feature_name = f'Mod_{feature}'
        data.loc[:, new_feature_name] = data['Planetary_Scaled'] * data[feature]
        modulation_feature_names.append(new_feature_name)
    
    feature_names = original_feature_names + modulation_feature_names

    # Print statement now shows 32 basic features (30 original + 1 Diff + 1 Accel)
    print(f"Number of Basic Features (including Diff and Accel): {len(original_feature_names)}") 
    # Total features should now be 64 (32 + 32)
    print(f"Total Features: {len(feature_names)} (Full Frequency Comb + Velocity + Acceleration)")
    
    # --- 3. Data Split and Transformation (For Training) ---
    
    # Only use rows with valid residuals for training and testing
    data_clean = data.dropna(subset=[target_residual_column] + feature_names)
    
    X = data_clean[feature_names]
    Y_raw = data_clean[target_residual_column] # Y Raw values
    groups = data_clean['Date'].dt.year
    dates = data_clean['Date']
    
    train_mask = (dates >= train_start_date) & (dates <= train_end_date)
    test_start_date_dt = pd.to_datetime(train_end_date) + pd.Timedelta(days=1)
    test_mask = (dates >= test_start_date_dt) & (dates <= test_end_date)
    
    X_train_raw = X[train_mask]
    Y_train_raw = Y_raw[train_mask]
    X_test_raw = X[test_mask]
    Y_test_raw = Y_raw[test_mask]
    groups_train = groups[train_mask]
    
    if len(X_train_raw) == 0 or len(X_test_raw) == 0:
        print("Error: Train or Test set is empty")
        return None, None

    # a. Fit X Transformer (StandardScaler)
    feature_scaler = StandardScaler()
    X_train = feature_scaler.fit_transform(X_train_raw)
    X_test = feature_scaler.transform(X_test_raw)

    # b. Fit Y Transformer (QuantileTransformer) - Key Fix
    print("Applying QuantileTransformer to stabilize target variable Y (Residual)...")
    target_transformer = QuantileTransformer(output_distribution='normal', random_state=42)
    Y_train = target_transformer.fit_transform(Y_train_raw.values.reshape(-1, 1)).flatten()
    Y_test = target_transformer.transform(Y_test_raw.values.reshape(-1, 1)).flatten()
    
    print(f"Data: Train {len(X_train)}, Test {len(X_test)}")
    
    # --- 4. Bayesian Optimization - LGBM (Strong Regularization) ---
    n_splits = max(2, min(5, groups_train.nunique()))
    gkf = GroupKFold(n_splits=n_splits)
    
    search_spaces = {
        'n_estimators': Integer(100, 800),
        'num_leaves': Integer(5, 30),
        'learning_rate': Real(1e-3, 0.1, 'log-uniform'),
        'reg_alpha': Real(0.1, 20.0, 'log-uniform'), 
        'reg_lambda': Real(0.1, 20.0, 'log-uniform'),
        'subsample': Real(0.7, 1.0, 'uniform'),
        'colsample_bytree': Real(0.7, 1.0, 'uniform')
    }
    
    lgb_model = lgb.LGBMRegressor(
        random_state=42, 
        n_jobs=1, 
        objective='regression_l1',
        force_row_wise=True, # Force row-wise to eliminate auto-detection warnings
        verbose=-1           # Suppress all LightGBM internal output (recommended)
    )

    opt_lgbm = BayesSearchCV(
        lgb_model, 
        search_spaces, 
        n_iter=100, 
        cv=gkf, 
        n_jobs=-1,
        random_state=42, 
        scoring='r2'
    )
    
    print("Training LGBM (Full Frequency Comb Version)...")
    opt_lgbm.fit(X_train, Y_train, groups=groups_train)
    
    # --- 5. Result Analysis ---
    best_model = opt_lgbm.best_estimator_
    
    # Predict on test set
    Y_pred_test = best_model.predict(X_test)
    
    # Calculate M_T_R2 (on transformed Y)
    final_r2_test = r2_score(Y_test, Y_pred_test) 
    
    # Get M_CV_R2 (Best CV Score on transformed Y)
    best_cv_score = opt_lgbm.best_score_
    
    print(f"\nLGBM Best CV R2 (M_CV_R2): {best_cv_score:.4f}")
    print(f"LGBM Test Set R2 (M_T_R2): {final_r2_test:.4f}")
    
    if final_r2_test > 0: # Check using the corrected variable name
        print("✅ Success! Model R2 is positive.")
    else:
        print("❌ R2 is still negative.")
        
    print(f"Best Params: {opt_lgbm.best_params_}")
    
    # Feature Importance
    importances = pd.Series(best_model.feature_importances_, index=feature_names)
    print("\nFeature Importance (Top 20):")
    top_features = importances.sort_values(ascending=False).head(20)    
    print(top_features.to_string())
    
    mod_features_in_top = [f for f in top_features.index if f.startswith('Mod_')]
    print(f"\nIn Top 20, {len(mod_features_in_top)} are modulation features.")
    
    # --- 6. New Functionality: Save Model ---
    print("\nSaving model and Scalers...")
    # Updated to use model_save_dir
    model_path = os.path.join(model_save_dir, f"{model_name}_best_lgbm.joblib")
    fscaler_path = os.path.join(model_save_dir, f"{model_name}_feature_scaler.joblib")
    tscaler_path = os.path.join(model_save_dir, f"{model_name}_target_transformer.joblib")
    
    joblib.dump(best_model, model_path)
    joblib.dump(feature_scaler, fscaler_path)
    joblib.dump(target_transformer, tscaler_path)
    print(f"Model saved: {model_path}")

    # --- 7. New Functionality: Generate Full Cycle Predictions ---
    print("Generating full cycle (to 2050) predictions...")
    
    # Prepare X features for full cycle
    # Must fill NaNs in features (e.g., if Planetary_Scaled is NaN at the end)
    X_full = data[feature_names].fillna(0)    
    
    X_full_scaled = feature_scaler.transform(X_full)
    Y_pred_full_transformed = best_model.predict(X_full_scaled)
    
    # Inverse transform back to original scale
    Y_pred_full_raw = target_transformer.inverse_transform(Y_pred_full_transformed.reshape(-1, 1)).flatten()
    
    # Create prediction column and secondary residual column
    pred_col_name = f'LGBM_Pred_{model_name}'
    resid_col_name = f'LGBM_Resid_{model_name}'
    
    data[pred_col_name] = Y_pred_full_raw
    # Original Residual - LGBM Predicted Residual = Secondary Residual
    data[resid_col_name] = data[target_residual_column] - data[pred_col_name]
    
    # Prepare stats dictionary
    stats_dict = {
        'model': model_name,
        # Added M_CV_R2 and M_T_R2
        'M_T_R2': final_r2_test,
        'M_CV_R2': best_cv_score,
        'n_features': len(feature_names),
        'top_20_mod_count': len(mod_features_in_top),
        'best_params': opt_lgbm.best_params_,
        'training_time': time.time() - start_time
    }

    # Return DataFrame with new columns (only necessary columns) and stats results
    return data[['Date', pred_col_name, resid_col_name]], stats_dict


# Main Execution - Run LGBM Full Frequency Comb Version
if __name__ == "__main__":
    model_splits = {
        'M8+2': '1996-08-01', 
        'M8+3': '1986-08-01',
        'M0+3': '1986-09-01', 
        'M0+2': '1996-08-01'
    }
    
    print("=== Run LGBM (Full Frequency Comb Version: B0 + 22y, QBO, 1y, 154d, 25-30d) ===")
    print("Strategy: Physical Full Frequency Comb + 11-year Modulation + Target Stabilization + Strong Regularization")
    print("New: Save model and full cycle secondary residuals")

    # Directory for saving models
    MODEL_SAVE_DIR = "../../results/05_p_m_a_model/m_model_4/LGBM_Models2" 
    
    # --- New: Load and Prepare Data Outside Loop ---
    try:
        data_source_file = '../../results/05_p_m_a_model/p_model_4/residual/Summary_Fit_Results.csv'
        print(f"\nLoading source data: {data_source_file}")
        full_data_to_save = pd.read_csv(data_source_file, parse_dates=['Date'])
    except Exception as e:
        print(f"Fatal Error: Cannot read {data_source_file}. Error: {e}")
        exit()

    # Check and extend dates to 2050
    max_date_in_file = full_data_to_save['Date'].max()
    target_date = pd.to_datetime('2050-12-31')
    
    if max_date_in_file < target_date:
        print(f"Warning: Source data max date is {max_date_in_file}.")
        print(f"Warning: Extending date range to {target_date}.")
        print("Warning: Please ensure 'Fit_SSN' (Planetary Fit) column in source file is extrapolated to 2050, otherwise predictions will be inaccurate!")
        
        all_dates_df = pd.DataFrame({'Date': pd.date_range(
            start=full_data_to_save['Date'].min(), 
            end=target_date, 
            freq='D'
        )})
        full_data_to_save = pd.merge(all_dates_df, full_data_to_save, on='Date', how='left')

    # To save stats results
    results_stats_list = []
    
    # Loop through each model
    for model_name, split_date in model_splits.items():
        print(f"\n{'='*50}")
        print(f"Processing Model: {model_name} (Train End: {split_date})")
        print(f"{'='*50}")
        
        # Pass full data
        result_df, stats_dict = run_lgbm_full_comb_validation(
            data_full=full_data_to_save.copy(), # Pass a copy of data
            model_name=model_name,
            train_start_date='1855-12-02',
            train_end_date=split_date,
            test_end_date='2019-11-30',
            model_save_dir=MODEL_SAVE_DIR # Pass the directory explicitly
        )
        
        if result_df is not None and stats_dict is not None:
            results_stats_list.append(stats_dict)
            # Merge this model's results back to main DataFrame
            full_data_to_save = pd.merge(full_data_to_save, result_df, on='Date', how='left')
    
    # --- New: Save CSV containing all results ---
    if results_stats_list:
        output_filename =  "../../results/05_p_m_a_model/m_model_4/LGBM_results2.csv"
        print(f"\n{'='*60}")
        print(f"Saving secondary fit and residuals for all models to: {output_filename}")
        try:
            full_data_to_save.to_csv(output_filename, index=False, float_format='%.6f')
            print(f"Save successful. File contains {len(full_data_to_save)} rows.")
        except Exception as e:
            print(f"Failed to save CSV: {e}")
        
        # --- Print Summary Report (Using stats data) ---
        results_to_print = []
        for r in results_stats_list:
            results_to_print.append({
                'model': r['model'],
                # Correction: Use new key name M_T_R2
                'M_T_R2': r['M_T_R2'], 
                # Correction: Add M_CV_R2 for easy viewing
                'M_CV_R2': r['M_CV_R2'], 
                'n_features': r['n_features'],
                'top_20_mod': r['top_20_mod_count'],
                'time(s)': r['training_time']
            })
            
        results_df = pd.DataFrame(results_to_print)
        print("\n" + "="*60)
        print("LGBM Full Frequency Comb Version Summary")
        print("="*60)
    
        # Print key columns
        print(results_df[['model', 'M_CV_R2', 'M_T_R2', 'n_features', 'top_20_mod', 'time(s)']].to_string(index=False, float_format="%.4f"))
        
        # Calculate mean using M_T_R2
        avg_r2 = results_df['M_T_R2'].mean()
        print(f"\nAverage Test R2 (M_T_R2): {avg_r2:.4f}")
        
        # Check positive/negative using M_T_R2
        positive_r2 = results_df[results_df['M_T_R2'] > 0]
        if len(positive_r2) == len(results_df):
            print("✅✅✅ Huge Success: All models have positive Test R2!")
        elif len(positive_r2) > 0:
            print(f"✅ Success: {len(positive_r2)} models obtained positive Test R2.")
        else:
            print("❌ Failure: All models still have negative Test R2.")
        
        if results_df['top_20_mod'].mean() >= 10: 
             print("💡 Insight: Modulation features (Mod_) play a significant role in the models.")

    else:
        print("\nAll models failed.")

LightGBM imported successfully.
=== Run LGBM (Full Frequency Comb Version: B0 + 22y, QBO, 1y, 154d, 25-30d) ===
Strategy: Physical Full Frequency Comb + 11-year Modulation + Target Stabilization + Strong Regularization
New: Save model and full cycle secondary residuals

Loading source data: ../../results/05_p_m_a_model/p_model_4/residual/Summary_Fit_Results.csv

Processing Model: M8+2 (Train End: 1996-08-01)

=== LGBM Final Full Frequency Comb Version: M8+2 ===
1. Generating B0 Ephemeris Features + Full Frequency Comb Features (Full Cycle)...
2. Generating modulation features (Full Cycle)...
... Adding 'Planetary_Scaled_Diff' (Velocity) feature...
... Adding 'Planetary_Scaled_Accel' (Acceleration) feature...
Number of Basic Features (including Diff and Accel): 32
Total Features: 64 (Full Frequency Comb + Velocity + Acceleration)
Applying QuantileTransformer to stabilize target variable Y (Residual)...
Data: Train 51378, Test 8521
Training LGBM (Full Frequency Comb Version)...

LGBM Bes